In [90]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [43]:
my_file = pd.read_excel('dataset/Data_Train.xlsx')
my_file.head(10)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
5,SpiceJet,24/06/2019,Kolkata,Banglore,CCU → BLR,09:00,11:25,2h 25m,non-stop,No info,3873
6,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,18:55,10:25 13 Mar,15h 30m,1 stop,In-flight meal not included,11087
7,Jet Airways,01/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:00,05:05 02 Mar,21h 5m,1 stop,No info,22270
8,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:55,10:25 13 Mar,25h 30m,1 stop,In-flight meal not included,11087
9,Multiple carriers,27/05/2019,Delhi,Cochin,DEL → BOM → COK,11:25,19:15,7h 50m,1 stop,No info,8625


### Sort by Date_of_Journey

In [44]:
dataset = my_file.sort_values(by=['Date_of_Journey'])
dataset.head(3)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
6024,Air India,01/03/2019,Banglore,New Delhi,BLR → MAA → DEL,11:50,08:55 02 Mar,21h 5m,1 stop,1 Long layover,14594
2405,Jet Airways,01/03/2019,Banglore,New Delhi,BLR → BOM → DEL,14:05,07:40 02 Mar,17h 35m,1 stop,1 Long layover,22270


### Check for Null values

In [46]:
dataset.isna().any()

Airline            False
Date_of_Journey    False
Source             False
Destination        False
Route               True
Dep_Time           False
Arrival_Time       False
Duration           False
Total_Stops         True
Additional_Info    False
Price              False
dtype: bool

In [49]:
#Checking for null values in Route and Total_Stops column
null_in_route = dataset[dataset['Route'].isnull()]
null_in_total_stops = dataset[dataset['Total_Stops'].isnull()]
null_in_route

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


In [50]:
null_in_total_stops

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


So both the column with missing value was the same row. 

In [51]:
#Delete the above row by using the index value
delete_row = dataset[dataset['Total_Stops'].isnull()].index
print(delete_row)
dataset = dataset.drop(delete_row)

Int64Index([9039], dtype='int64')


In [52]:
#Checking for null values
print(dataset.isnull().values.any())

False


In [54]:
#Get the total rows and column in the dataset
print("Train file shape", dataset.shape)
#Checking the types of column
print(dataset.dtypes)

Train file shape (10682, 11)
Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object


In [55]:
#Convert Date_of_Journey, Arrival_Time to datetime format
dataset['Date_of_Journey'] = pd.to_datetime(dataset['Date_of_Journey'])
dataset['Arrival_Time'] = pd.to_datetime(dataset['Arrival_Time'])

In [61]:
#Checking the 'Duration' column to see if there is any row which takes only minutes to travel. This may be an outlier
def check_duration_col(row):
    if 'h' not in row:
        print(row)
dataset['Duration'].apply(check_duration_col)

5m


9848     None
6024     None
2405     None
10383    None
8308     None
         ... 
2875     None
2874     None
2873     None
6479     None
7297     None
Name: Duration, Length: 10682, dtype: object

In [62]:
#Since duration of travel cannot be 5 mins hence removing the row.
dataset = dataset.drop(dataset[dataset['Duration'] == '5m'].index)

In [63]:
#Converting Duration column from hours and minutes to minutes only
import re
def convert_hours_to_mins(row):
    row = row.split(' ')
    match_hr = re.match(r"([0-9]+)([a-z]+)", row[0], re.I)
    if len(row) > 1:
        match_min = re.match(r"([0-9]+)([a-z]+)", row[1], re.I)
        mins = match_min.groups()
    else:
        mins = [0]
    if match_hr:
        hr = match_hr.groups()
    hr_to_min = int(hr[0]) * 60
    return int(hr_to_min) + int(mins[0])

In [64]:
dataset['Duration'] = dataset['Duration'].apply(convert_hours_to_mins)

In [65]:
dataset.head(3)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9848,Air India,2019-01-03,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,2020-03-02 23:55:00,2345,2 stops,No info,17135
6024,Air India,2019-01-03,Banglore,New Delhi,BLR → MAA → DEL,11:50,2020-03-02 08:55:00,1265,1 stop,1 Long layover,14594
2405,Jet Airways,2019-01-03,Banglore,New Delhi,BLR → BOM → DEL,14:05,2020-03-02 07:40:00,1055,1 stop,1 Long layover,22270


In [66]:
#Remove stops from Total_stops
def remove_stops(row):
    if row == 'non-stop':
        row = 0
    else:
        row = row.split(' ')[0]
    return row
dataset['Total_Stops'] = dataset['Total_Stops'].apply(remove_stops)

In [67]:
dataset['Additional_Info'] = dataset['Additional_Info'].str.lower()
dataset['Additional_Info'].value_counts()

no info                         8346
in-flight meal not included     1982
no check-in baggage included     320
1 long layover                    19
change airports                    7
business class                     4
red-eye flight                     1
2 long layover                     1
1 short layover                    1
Name: Additional_Info, dtype: int64

In [68]:
#Remove colon from Departure Time
def remove_colon(row):
    if ':' in str(row):
        row = row.replace(':', '')
    return int(row)

dataset['Dep_Time'] = dataset['Dep_Time'].apply(remove_colon)

In [74]:
#Saving the dataframe to xlsx format
dataset.to_excel('dataset/final_data.xlsx', index=0)

In [76]:
#Selecting features from dataframe to prepare data for training
dataframe_x = dataset[['Airline', 'Source', 'Destination', 'Dep_Time', 'Duration', 'Total_Stops', 'Additional_Info']]
dataframe_y = dataset[['Price']]

In [77]:
#Converting categorical columns to one-hot encoding 
dataframe_x = pd.get_dummies(dataframe_x, columns=['Source', 'Additional_Info', 'Airline', 'Destination'])

In [78]:
dataframe_x.head(5)

,Dep_Time,Duration,Total_Stops,Source_Banglore,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Additional_Info_1 long layover,Additional_Info_1 short layover,...,Airline_SpiceJet,Airline_Trujet,Airline_Vistara,Airline_Vistara Premium economy,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
9848,850,2345,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6024,1150,1265,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2405,1405,1055,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
10383,700,1325,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8308,1825,175,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [79]:
#Splitting the data into train, validation and test 
train_dataframe_x = dataframe_x[:7000]
train_dataframe_y = dataframe_y[:7000]
val_dataframe_x = dataframe_x[7000:8500]
val_dataframe_y = dataframe_y[7000:8500]
test_dataframe_x = dataframe_x[8500:]
test_dataframe_y = dataframe_y[8500:]
print(train_dataframe_x.shape, val_dataframe_x.shape, test_dataframe_x.shape)
print(train_dataframe_y.shape, val_dataframe_y.shape, test_dataframe_y.shape)

(7000, 35) (1500, 35) (2181, 35)
(7000, 1) (1500, 1) (2181, 1)


In [80]:
#Standardizing the datasets
scaler = preprocessing.StandardScaler()
train_dataframe_x = scaler.fit_transform(train_dataframe_x)
val_dataframe_x = scaler.transform(val_dataframe_x)
test_dataframe_x = scaler.transform(test_dataframe_x)

In [81]:
#Finding the best hyperparameters to train the Random Forest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import datetime
param_grid = {
 'max_depth': [4, 8, 16, 32],
 'n_estimators': [1, 2, 5, 10, 50, 100, 200]
}
t1 = datetime.datetime.now()
rf = RandomForestRegressor(n_jobs=-1)
clf = GridSearchCV(estimator = rf, param_grid = param_grid)
clf.fit(val_dataframe_x,val_dataframe_y)
print("time required = ", datetime.datetime.now() - t1)

C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataCon

C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataCon

C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataCon

C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataCon

C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_validation.py:531: DataCon

time required =  0:00:30.305977


C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


In [82]:
clf.best_params_

{'max_depth': 16, 'n_estimators': 50}

## Random Forest Model

In [83]:
rf_model = RandomForestRegressor(max_depth = clf.best_params_['max_depth'], n_estimators=clf.best_params_['n_estimators'])
rf_model.fit(train_dataframe_x, train_dataframe_y)

C:\Users\USER\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(max_depth=16, n_estimators=50)

In [93]:
#Getting scores
print("Train Score", rf_model.score(train_dataframe_x, train_dataframe_y))
print("Test Score", rf_model.score(test_dataframe_x, test_dataframe_y))

Train Score 0.8584312057006556
Test Score 0.8074256564973739


In [86]:
#making prediction on test data
y_pred = rf_model.predict(test_dataframe_x)

In [87]:
#Calculating the metrics
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(test_dataframe_y, y_pred))
print('MSE:', metrics.mean_squared_error(test_dataframe_y, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(test_dataframe_y, y_pred)))
print("R-squared score:", metrics.r2_score(test_dataframe_y, y_pred))

MAE: 1023.5157312927405
MSE: 3116016.447744766
RMSE: 1765.2241919214584
R-squared score: 0.8074256564973739


## XGBoost Model

In [98]:
#Using XGBoost Regressor
#Finding best parameters
params = {'max_depth': [1, 2, 3, 4, 5], 'learning_rate': [0.01, 0.05, 0.1], 'n_estimators': [20, 50, 100, 200, 300, 400, 500], 'reg_lambda': [0.001, 0.1, 1.0, 10.0, 100.0]}
xgb_model = RandomizedSearchCV(XGBRegressor(), params, n_iter=20, cv=10, n_jobs=-1)
xgb_model.fit(train_dataframe_x, train_dataframe_y)

RandomizedSearchCV(cv=10,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100,...e,
                                          num_parallel_tree=None,
                                          random_state=None, reg_alpha=None,
                                          reg_

In [100]:
#Getting scores
print("Train Score", xgb_model.score(train_dataframe_x, train_dataframe_y))
print("Test Score", xgb_model.score(test_dataframe_x, test_dataframe_y))

Train Score 0.8507264735798687
Test Score 0.7923837391518385


In [102]:
#making prediction on test data
y_pred_xgb = xgb_model.predict(test_dataframe_x)

In [103]:
#Calculating the metrics
print('MAE:', metrics.mean_absolute_error(test_dataframe_y, y_pred_xgb))
print('MSE:', metrics.mean_squared_error(test_dataframe_y, y_pred_xgb))
print('RMSE:', np.sqrt(metrics.mean_squared_error(test_dataframe_y, y_pred_xgb)))
print("R-squared score:", metrics.r2_score(test_dataframe_y, y_pred_xgb))

MAE: 1058.219327833061
MSE: 3359407.446783361
RMSE: 1832.8686387145592
R-squared score: 0.7923837391518385
